In [ ]:
def create_model(hp):
    model = models.Sequential()
    
    # Convolutional layers
    for i in range(hp['num_layers']):
        if i == 0:
            model.add(layers.Conv2D(hp['filters'][i], hp['kernel_size'][i], activation='relu', input_shape=(48, 48, 3)))
        else:
            model.add(layers.Conv2D(hp['filters'][i], hp['kernel_size'][i], activation='relu'))
        model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Flatten())
    
    # Dense layers
    model.add(layers.Dense(hp['dense_units'], activation='relu'))
    model.add(layers.Dense(6, activation='softmax'))  # Assuming 6 classes
    
    # Compile the model
    model.compile(optimizer=optimizers.Adam(learning_rate=hp['learning_rate']),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model



def objective(trial):
    # Define hyperparameters to optimize
    batch_size = trial.suggest_int('batch_size', 16, 64)
    num_layers = trial.suggest_int('num_layers', 1, 3)
    filters = [trial.suggest_int(f'filters_{i}', 16, 128) for i in range(num_layers)]
    kernel_size = [trial.suggest_categorical(f'kernel_size_{i}', [(3, 3), (5, 5)]) for i in range(num_layers)]
    dense_units = trial.suggest_int('dense_units', 64, 512)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
    
    epochs = 3 

    
    hp = {
        'batch_size': batch_size,
        'num_layers': num_layers,
        'filters': filters,
        'kernel_size': kernel_size,
        'dense_units': dense_units,
        'learning_rate': learning_rate,
        'epochs': epochs
    }

   
    model = create_model(hp)

   
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True
    )

   
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=hp['epochs'],
        batch_size=hp['batch_size'],
        callbacks=[early_stopping]
    )

   
    val_loss, val_acc = model.evaluate(val_ds)
    return val_acc 


study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=50)

# Get the best trial
best_trial = study.best_trial

# Print the best trial results
print(f"Best trial parameters: {best_trial.params}")
print(f"Best trial value: {best_trial.value}")

# Optional: Train the final model with the best hyperparameters
best_hp = best_trial.params
final_model = create_model(best_hp)
final_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=best_hp['epochs'],
    batch_size=best_hp['batch_size']
)
